In [6]:
from introspect import introspect_schema
from base import graphql_request, introspect_schema, inspect_with_langchain
from typing import Optional, Union
import openai
import logging
from llama_index.agent import OpenAIAgent
import requests
from llama_index.bridge.langchain import FunctionMessage
from langchain.agents import initialize_agent, AgentType
from llama_hub.tools.graphql.base import GraphQLToolSpec
import json
from typing import Sequence
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory
from llama_index.tools import BaseTool, FunctionTool
from langchain.tools import BaseTool, StructuredTool
from llama_index.llms import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os

openai_key = os.getenv("OPENAI_API_KEY")
class MyOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: ChatOpenAI = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0613", openai_api_key = openai_key),
        chat_history: ChatMessageHistory = ChatMessageHistory(),
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history.clear()

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.add_user_message(message)
        functions = [tool.metadata.to_openai_function() for _, tool in self._tools.items()]

        ai_message = self._llm.predict_messages(chat_history.messages, functions=functions)
        chat_history.add_message(ai_message)

        function_call = ai_message.additional_kwargs.get("function_call", None)
        if function_call is not None:
            function_message = self._call_function(function_call)
            chat_history.add_message(function_message)
            ai_message = self._llm.predict_messages(chat_history.messages)
            chat_history.add_message(ai_message)

        return ai_message.content

    def _call_function(self, function_call: dict) -> FunctionMessage:
        tool = self._tools[function_call["name"]]
        output = tool(**json.loads(function_call["arguments"]))
        return FunctionMessage(
            name=function_call["name"],
            content=str(output), 
        )





def inspect_with_llama(prompt, openai_key):
    query_tool = FunctionTool.from_defaults(fn=graphql_request)
    introspect_tool = FunctionTool.from_defaults(fn=introspect_schema)
   
    #agent = MyOpenAIAgent(tools=[query_tool, introspect_tool])
    llm = OpenAI(model="gpt-3.5-turbo-0613")

    agent = OpenAIAgent.from_tools(
    [query_tool, introspect_tool], llm=llm, verbose=True
)
    openai.api_key = openai_key
    print(agent.chat(prompt))


prompt = '''query MyQuery {
  swaps(limit: 10, orderBy: id_ASC) {
    id
    amount1
    amount0
    blockNumber
  }
}
query and provide consice summary of the data with values'''
inspect_with_llama(prompt, openai_key)

=== Calling Function ===
Calling function: graphql_request with args: {
  "query": "query MyQuery {\n  swaps(limit: 10, orderBy: id_ASC) {\n    id\n    amount1\n    amount0\n    blockNumber\n  }\n}"
}
{"data":{"swaps":[{"id":"0012377980-000072-415a3","amount1":"1000000000000000000","amount0":"-3759611132739177664039","blockNumber":12377980},{"id":"0012377980-000310-415a3","amount1":"-200007821","amount0":"57521033320909789","blockNumber":12377980},{"id":"0012377980-000311-415a3","amount1":"200007821","amount0":"-200000000000000000000","blockNumber":12377980},{"id":"0012377982-000065-28357","amount1":"1000000000000000000","amount0":"-11659453568618418539","blockNumber":12377982},{"id":"0012377983-000063-251e8","amount1":"1000000000000000000","amount0":"-3740413022613129545357","blockNumber":12377983},{"id":"0012377984-000003-64262","amount1":"1000000000000000000","amount0":"-3721361588082517023844","blockNumber":12377984},{"id":"0012377984-000172-64262","amount1":"-301897198105914826","

In [7]:
prompt = '''explore the schema and based on fields available in swaps, propose metrics that can be used to evaluate swap intensity on uniswap'''

print(inspect_with_langchain(prompt, openai_key))



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "introspect_schema",
  "action_input": {}
}
```

{'__schema': {'types': [{'kind': 'OBJECT', 'name': 'Query', 'description': None, 'enumValues': None, 'fields': [{'name': 'pools', 'args': [{'name': 'where'}, {'name': 'orderBy'}, {'name': 'offset'}, {'name': 'limit'}], 'type': {'kind': 'NON_NULL', 'name': None, 'ofType': {'name': None}}}, {'name': 'poolById', 'args': [{'name': 'id'}], 'type': {'kind': 'OBJECT', 'name': 'Pool', 'ofType': None}}, {'name': 'poolsConnection', 'args': [{'name': 'orderBy'}, {'name': 'after'}, {'name': 'first'}, {'name': 'where'}], 'type': {'kind': 'NON_NULL', 'name': None, 'ofType': {'name': 'PoolsConnection'}}}, {'name': 'swaps', 'args': [{'name': 'where'}, {'name': 'orderBy'}, {'name': 'offset'}, {'name': 'limit'}], 'type': {'kind': 'NON_NULL', 'name': None, 'ofType': {'name': None}}}, {'name': 'swapById', 'args': [{'name': 'id'}], 'type': {'kind': 'OBJECT', 'name': 'Swap', 'ofType': None}}, {